# CMSC Interactive Fiction Storytelling - DnD Project  
### LLM TO DETECTION PLAYER INTENT BASED ON PLAYER NARRATIVES AND GAME STATE
#### Team Members, Arya Honraopatil, Saksham Kumar Sharma, and Patty Delafuente  

Ablation study -  Base model eval on M1 and M2 datasets  
Refer to the 691-Dnd_project_LlamTraining_m1 and 

In [5]:
import os
import json
import numpy as np
from rouge_score import rouge_scorer, scoring

In [1]:
%%bash
MODEL="models/llama3_1_8b_instruct.nemo"
#MODEL="/workspace/llama-3_1-8b-instruct-nemo_v1.0/llama3_1_8b_instruct.nemo"



TEST_DS="[./data/fireball_test_finetuning-n128_m1.jsonl]" # Smaller test split
# TEST_DS="[./curated-data/law-qa-test_preprocessed.jsonl]" # Full test set
TEST_NAMES="[fireball]"

TP_SIZE=1
PP_SIZE=1

# This is where your LoRA checkpoint was saved
#PATH_TO_TRAINED_MODEL="results/Meta-llama3.1-8B-Instruct-fb_m1/checkpoints/megatron_gpt_peft_lora_tuning.nemo"

# The generation run will save the generated outputs over the test dataset in a file prefixed like so
OUTPUT_PREFIX="fb_lora_base"
#global batch size (1) is not divisible by micro batch size (1) times data parallel size (2)
torchrun --nproc_per_node=1
python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py \
    model.restore_from_path=${MODEL} \
    model.peft.restore_from_path=null \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    model.data.test_ds.file_names=${TEST_DS} \
    model.data.test_ds.names=${TEST_NAMES} \
    model.data.test_ds.global_batch_size=16 \
    model.data.test_ds.micro_batch_size=1 \
    model.data.test_ds.tokens_to_generate=25 \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    inference.greedy=True  \
    model.data.test_ds.output_file_path_prefix=${OUTPUT_PREFIX} \
    model.data.test_ds.write_predictions_to_file=True \
    model.data.test_ds.truncation_field="null" \
    model.data.test_ds.add_bos=False \
    model.data.test_ds.add_eos=True \
    model.data.test_ds.add_sep=False \
    model.data.test_ds.label_key="output" \
    model.data.test_ds.prompt_template="\{input\}\ \{output\}"

usage: torchrun [-h] [--nnodes NNODES] [--nproc-per-node NPROC_PER_NODE]
                [--rdzv-backend RDZV_BACKEND] [--rdzv-endpoint RDZV_ENDPOINT]
                [--rdzv-id RDZV_ID] [--rdzv-conf RDZV_CONF] [--standalone]
                [--max-restarts MAX_RESTARTS]
                [--monitor-interval MONITOR_INTERVAL]
                [--start-method {spawn,fork,forkserver}] [--role ROLE] [-m]
                [--no-python] [--run-path] [--log-dir LOG_DIR] [-r REDIRECTS]
                [-t TEE] [--local-ranks-filter LOCAL_RANKS_FILTER]
                [--node-rank NODE_RANK] [--master-addr MASTER_ADDR]
                [--master-port MASTER_PORT] [--local-addr LOCAL_ADDR]
                training_script ...
torchrun: error: the following arguments are required: training_script, training_script_args
`zarr` distributed checkpoint backend is deprecated. Please switch to PyTorch Distributed format (`torch_dist`).
[NeMo W 2024-12-12 22:15:56 nemo_logging:349] /usr/local/lib/python3.10/d

[NeMo I 2024-12-12 22:15:56 megatron_gpt_generate:125] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-12 22:15:56 megatron_gpt_generate:126] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 20000
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.test_ds.metric.name}
        save_top_k: 1
        mode: max
        save_nemo_o

[NeMo W 2024-12-12 22:15:56 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2024-12-12 22:16:06 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-12 22:16:06 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-12 22:16:06 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: moe_extended_tp in its cfg. Add this key to cfg or config_mapping to make to make it 

[NeMo I 2024-12-12 22:16:06 megatron_init:269] Rank 0 has data parallel group : [0]
[NeMo I 2024-12-12 22:16:06 megatron_init:275] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2024-12-12 22:16:06 megatron_init:280] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2024-12-12 22:16:06 megatron_init:283] Ranks 0 has data parallel rank: 0
[NeMo I 2024-12-12 22:16:06 megatron_init:291] Rank 0 has context parallel group: [0]
[NeMo I 2024-12-12 22:16:06 megatron_init:294] All context parallel group ranks: [[0]]
[NeMo I 2024-12-12 22:16:06 megatron_init:295] Ranks 0 has context parallel rank: 0
[NeMo I 2024-12-12 22:16:06 megatron_init:302] Rank 0 has model parallel group: [0]
[NeMo I 2024-12-12 22:16:06 megatron_init:303] All model parallel group ranks: [[0]]
[NeMo I 2024-12-12 22:16:06 megatron_init:312] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-12-12 22:16:06 megatron_init:316] All tensor model parallel group ranks: 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Testing DataLoader 0:   0%|          | 0/8 [00:00<?, ?it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  12%|█▎        | 1/8 [01:02<07:20,  0.02it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  25%|██▌       | 2/8 [01:40<05:00,  0.02it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  38%|███▊      | 3/8 [02:49<04:42,  0.02it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  50%|█████     | 4/8 [03:52<03:52,  0.02it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  62%|██████▎   | 5/8 [05:29<03:17,  0.02it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  75%|███████▌  | 6/8 [06:1

[NeMo W 2024-12-12 22:24:58 megatron_gpt_sft_model:642] No training data found, reconfiguring microbatches based on validation batch sizes.


setting number of micro-batches to constant 16


[NeMo W 2024-12-12 22:24:58 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-12 22:24:58 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('test_loss_fireball', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-12 22:24:58 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('test_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    


Testing DataLoader 0: 100%|██████████| 8/8 [08:22<00:00,  0.02it/s]
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     4.491147518157959     │
│    test_loss_fireball     │     4.491147518157959     │
│         val_loss          │     4.491147518157959     │
└───────────────────────────┴───────────────────────────┘


In [2]:
# Take a look at predictions
!head -n1  fb_lora_base_test_fireball_inputs_preds_labels.jsonl

{"input": "Current Player: {actions: None, attacks: Beak, Talons, class: None, controller_id: 981381483504357172, description: None, effects: buff, hp: <165/248 HP; Injured>, name: air monster, race: Roc, spells: }[SEP] Utterances : [Keeps shredding minion and will move 60ft up to be at 160total][SEP] \nAvrae Command:", "pred": " /attack @air monster with Beak\nAvraa Response: [SEP] Current Player: {actions: None,", "label": " [!i a bite -t min][SEP]"}


In [3]:
def compute_rouge(input_file: str) -> dict:
    ROUGE_KEYS = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    scorer = rouge_scorer.RougeScorer(ROUGE_KEYS, use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()
    lines = [json.loads(line) for line in open(input_file)]
    num_response_words = []
    num_ref_words = []
    for idx, line in enumerate(lines):
        prompt = line['input']
        response = line['pred']
        answer = line['label']
        scores = scorer.score(response, answer)
        aggregator.add_scores(scores)
        num_response_words.append(len(response.split()))
        num_ref_words.append(len(answer.split()))

    result = aggregator.aggregate()
    rouge_scores = {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
    print(rouge_scores)
    print(f"Average and stddev of response length: {np.mean(num_response_words):.2f}, {np.std(num_response_words):.2f}")
    print(f"Average and stddev of ref length: {np.mean(num_ref_words):.2f}, {np.std(num_ref_words):.2f}")

    return rouge_scores

In [13]:
compute_rouge("./fb_lora_m1_test_fireball_inputs_preds_labels.jsonl")

{'rouge1': 45.4714, 'rouge2': 13.6678, 'rougeL': 45.0966, 'rougeLsum': 44.9783}
Average and stddev of response length: 4.75, 1.97
Average and stddev of ref length: 12.28, 22.03


{'rouge1': 45.4714, 'rouge2': 13.6678, 'rougeL': 45.0966, 'rougeLsum': 44.9783}

In [6]:
compute_rouge("./fb_lora_base_test_fireball_inputs_preds_labels.jsonl")

{'rouge1': 13.2542, 'rouge2': 3.4214, 'rougeL': 12.4401, 'rougeLsum': 12.1552}
Average and stddev of response length: 11.42, 2.43
Average and stddev of ref length: 12.28, 22.03


{'rouge1': 13.2542, 'rouge2': 3.4214, 'rougeL': 12.4401, 'rougeLsum': 12.1552}

In [8]:
%%bash
MODEL="models/llama3_1_8b_instruct.nemo"
#MODEL="/workspace/llama-3_1-8b-instruct-nemo_v1.0/llama3_1_8b_instruct.nemo"



TEST_DS="[./data/fireball_test_finetuning-n128_m2.jsonl]" # Smaller test split
# TEST_DS="[./curated-data/law-qa-test_preprocessed.jsonl]" # Full test set
TEST_NAMES="[fireball]"

TP_SIZE=1
PP_SIZE=1

# This is where your LoRA checkpoint was saved
PATH_TO_TRAINED_MODEL="results/Meta-llama3.1-8B-Instruct-fb_m2/checkpoints/megatron_gpt_peft_lora_tuning.nemo"

# The generation run will save the generated outputs over the test dataset in a file prefixed like so
OUTPUT_PREFIX="fb_lora_base_m2"
#global batch size (1) is not divisible by micro batch size (1) times data parallel size (2)
torchrun --nproc_per_node=1
python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py \
    model.restore_from_path=${MODEL} \
    model.peft.restore_from_path=null \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    model.data.test_ds.file_names=${TEST_DS} \
    model.data.test_ds.names=${TEST_NAMES} \
    model.data.test_ds.global_batch_size=16 \
    model.data.test_ds.micro_batch_size=1 \
    model.data.test_ds.tokens_to_generate=25 \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    inference.greedy=True  \
    model.data.test_ds.output_file_path_prefix=${OUTPUT_PREFIX} \
    model.data.test_ds.write_predictions_to_file=True \
    model.data.test_ds.truncation_field="null" \
    model.data.test_ds.add_bos=False \
    model.data.test_ds.add_eos=True \
    model.data.test_ds.add_sep=False \
    model.data.test_ds.label_key="output" \
    model.data.test_ds.prompt_template="\{input\}\ \{output\}"

usage: torchrun [-h] [--nnodes NNODES] [--nproc-per-node NPROC_PER_NODE]
                [--rdzv-backend RDZV_BACKEND] [--rdzv-endpoint RDZV_ENDPOINT]
                [--rdzv-id RDZV_ID] [--rdzv-conf RDZV_CONF] [--standalone]
                [--max-restarts MAX_RESTARTS]
                [--monitor-interval MONITOR_INTERVAL]
                [--start-method {spawn,fork,forkserver}] [--role ROLE] [-m]
                [--no-python] [--run-path] [--log-dir LOG_DIR] [-r REDIRECTS]
                [-t TEE] [--local-ranks-filter LOCAL_RANKS_FILTER]
                [--node-rank NODE_RANK] [--master-addr MASTER_ADDR]
                [--master-port MASTER_PORT] [--local-addr LOCAL_ADDR]
                training_script ...
torchrun: error: the following arguments are required: training_script, training_script_args
`zarr` distributed checkpoint backend is deprecated. Please switch to PyTorch Distributed format (`torch_dist`).
[NeMo W 2024-12-12 22:32:01 nemo_logging:349] /usr/local/lib/python3.10/d

[NeMo I 2024-12-12 22:32:01 megatron_gpt_generate:125] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-12 22:32:02 megatron_gpt_generate:126] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 20000
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.test_ds.metric.name}
        save_top_k: 1
        mode: max
        save_nemo_o

[NeMo W 2024-12-12 22:32:02 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2024-12-12 22:32:07 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-12 22:32:07 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-12 22:32:07 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: moe_extended_tp in its cfg. Add this key to cfg or config_mapping to make to make it 

[NeMo I 2024-12-12 22:32:07 megatron_init:269] Rank 0 has data parallel group : [0]
[NeMo I 2024-12-12 22:32:07 megatron_init:275] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2024-12-12 22:32:07 megatron_init:280] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2024-12-12 22:32:07 megatron_init:283] Ranks 0 has data parallel rank: 0
[NeMo I 2024-12-12 22:32:07 megatron_init:291] Rank 0 has context parallel group: [0]
[NeMo I 2024-12-12 22:32:07 megatron_init:294] All context parallel group ranks: [[0]]
[NeMo I 2024-12-12 22:32:07 megatron_init:295] Ranks 0 has context parallel rank: 0
[NeMo I 2024-12-12 22:32:07 megatron_init:302] Rank 0 has model parallel group: [0]
[NeMo I 2024-12-12 22:32:07 megatron_init:303] All model parallel group ranks: [[0]]
[NeMo I 2024-12-12 22:32:07 megatron_init:312] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-12-12 22:32:07 megatron_init:316] All tensor model parallel group ranks: 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Testing DataLoader 0:   0%|          | 0/8 [00:00<?, ?it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  12%|█▎        | 1/8 [01:18<09:10,  0.01it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  25%|██▌       | 2/8 [02:43<08:11,  0.01it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  38%|███▊      | 3/8 [04:09<06:56,  0.01it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  50%|█████     | 4/8 [05:44<05:44,  0.01it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  62%|██████▎   | 5/8 [07:10<04:18,  0.01it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  75%|███████▌  | 6/8 [08:3

[NeMo W 2024-12-12 22:44:08 megatron_gpt_sft_model:642] No training data found, reconfiguring microbatches based on validation batch sizes.


setting number of micro-batches to constant 16


[NeMo W 2024-12-12 22:44:08 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-12 22:44:08 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('test_loss_fireball', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-12 22:44:08 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('test_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    


Testing DataLoader 0: 100%|██████████| 8/8 [11:31<00:00,  0.01it/s]
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     4.269781112670898     │
│    test_loss_fireball     │     4.269781112670898     │
│         val_loss          │     4.269781112670898     │
└───────────────────────────┴───────────────────────────┘


In [10]:
# Take a look at predictions
!head -n1  fb_lora_base_m2_test_fireball_inputs_preds_labels.jsonl

{"input": "You are a game agent in a DnD game. You will be provided with the current player, game state and the in game player utterance. Your task is to predict what action, attack or spell the play intends. Then provide the AVRAE command that will initiate the players intention in a discord DnD game. Return the avrae command. Here is the game state. [CLS] Current Player: {actions: None, attacks: Beak, Talons, class: None, controller_id: 981381483504357172, description: None, effects: buff, hp: <165/248 HP; Injured>, name: air monster, race: Roc, spells: }[SEP] Game State: Player: Minion () (<100/100 HP; Healthy> (+4 temp)) (Club) () Player: Fiona (Cleric 20) (<258/258 HP; Healthy>) (Akivean, 2-Handed Akivean, Crossbow, light, Unarmed Strike) (Beacon of Hope, Telekinesis, Dispel Magic, Revivify, Mending, Dispel Evil and Good, Warding Bond, Dimension Door, Guardian of Faith, Conjure Elemental, Mass Heal, Magic Missile, Sanctuary, True Seeing, Word of Radiance, Summon Celestial, Sleep, 

In [9]:
compute_rouge("./fb_lora_base_m2_test_fireball_inputs_preds_labels.jsonl")

{'rouge1': 8.0214, 'rouge2': 2.9196, 'rougeL': 7.6025, 'rougeLsum': 7.8644}
Average and stddev of response length: 14.60, 2.89
Average and stddev of ref length: 9.37, 17.10


{'rouge1': 8.0214, 'rouge2': 2.9196, 'rougeL': 7.6025, 'rougeLsum': 7.8644}